In [1]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "987f078e",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Gerando massa de dados de produção para simulação...\n",
      "50 ordens de produção simuladas foram geradas.\n",
      "\n",
      "PASSO 1: Aplicando regras de negócio (pré-processamento)...\n",
      "PASSO 2: Executando Estágio 1 (Planejamento e alocação diária)...\n",
      "PASSO 3: Executando Estágio 2 (Otimização da sequência de cada dia)...\n",
      "Otimização concluída.\n",
      "PASSO 4: Exportando resultados para o arquivo Excel...\n",
      "\n",
      "Aba 'Cronograma_Otimizado' gerada com sucesso.\n",
      "Aba 'Itens_Nao_Planejados' gerada com sucesso.\n",
      "\n",
      "Arquivo 'cronograma_otimizado_sprint4_documentado.xlsx' salvo com sucesso.\n",
      "\n",
      "Processo finalizado com sucesso em 0.09 segundos.\n"
     ]
    }
   ],
   "source": [
    "# -*- coding: utf-8 -*-\n",
    "\"\"\"\n",
    "================================================================================\n",
    "| SCRIPT DE OTIMIZAÇÃO DO SEQUENCIAMENTO DE PINTURA - NATHOR                   |\n",
    "================================================================================\n",
    "|                                                                              |\n",
    "| Versão: 3.0                         |\n",
    "| Autor: Manuel Finda                                                            |\n",
    "| Data: 2025-09-03                                                             |\n",
    "|                                                                              |\n",
    "|------------------------------------------------------------------------------|\n",
    "| DESCRIÇÃO GERAL                                                              |\n",
    "|------------------------------------------------------------------------------|\n",
    "| Este script constitui uma Prova de Conceito (POC) avançada para a otimização |\n",
    "| do sequenciamento de produção na área de pintura da Nathor. O sistema        |\n",
    "| utiliza um algoritmo híbrido (heurística + meta-heurística) para gerar um    |\n",
    "| cronograma de produção diário que busca o equilíbrio ótimo entre duas metas  |\n",
    "| principais:                                                                  |\n",
    "|                                                                              |\n",
    "| 1. RESPEITO ÀS PRIORIDADES DE NEGÓCIO: Garantir que as ordens de produção     |\n",
    "|    mais urgentes, com base no saldo de estoque e datas de entrega, sejam     |\n",
    "|    produzidas primeiro.                                                      |\n",
    "|                                                                              |\n",
    "| 2. MÁXIMA EFICIÊNCIA OPERACIONAL: Minimizar o tempo de máquina parada         |\n",
    "|    (setups), reduzindo o número de trocas de cor e de tipo de peça.          |\n",
    "|                                                                              |\n",
    "| O algoritmo considera múltiplas restrições do \"chão de fábrica\", como a      |\n",
    "| capacidade de fornecimento da metalurgia, o limite de armazenamento em       |\n",
    "| \"gaiolas\" e a disponibilidade de \"gancheiras\".                               |\n",
    "|                                                                              |\n",
    "|------------------------------------------------------------------------------|\n",
    "| FLUXO DE EXECUÇÃO DO SCRIPT                                                  |\n",
    "|------------------------------------------------------------------------------|\n",
    "| 1. Pré-processamento: Regras de negócio, como a antecipação de peças         |\n",
    "|    especiais, são aplicadas aos dados de entrada.                            |\n",
    "| 2. Estágio 1 (Planejamento): O algoritmo principal aloca as ordens de        |\n",
    "|    produção nos dias, respeitando todas as restrições de capacidade.        |\n",
    "| 3. Estágio 2 (Otimização): Para cada dia planejado, a sequência de produção  |\n",
    "|    é refinada pela meta-heurística Busca Tabu para minimizar os custos de    |\n",
    "|    setup.                                                                    |\n",
    "| 4. Exportação: O resultado final (cronograma otimizado e lista de itens     |\n",
    "|    rejeitados) é salvo em um arquivo Excel (.xlsx) para fácil análise.       |\n",
    "|                                                                              |\n",
    "|------------------------------------------------------------------------------|\n",
    "| DEPENDÊNCIAS                                                                 |\n",
    "|------------------------------------------------------------------------------|\n",
    "| - pandas: Para manipulação de dados e exportação para Excel.                 |\n",
    "| - openpyxl: Motor para a escrita de arquivos .xlsx pelo pandas.              |\n",
    "|   (Instalação: pip install pandas openpyxl)                                  |\n",
    "|                                                                              |\n",
    "================================================================================\n",
    "\"\"\"\n",
    "\n",
    "# --- 1. BIBLIOTECAS ---\n",
    "# Importação das bibliotecas necessárias para a execução do script.\n",
    "import pandas as pd\n",
    "from datetime import datetime, timedelta\n",
    "import copy\n",
    "import time\n",
    "import math\n",
    "import random\n",
    "\n",
    "# --- 2. CONFIGURAÇÃO E PARÂMETROS GLOBAIS ---\n",
    "# Nesta seção, definimos as constantes que governam as regras de negócio\n",
    "# e as capacidades da linha de produção. Alterar estes valores impacta\n",
    "# diretamente o resultado da simulação.\n",
    "\n",
    "# Custos de tempo para setups (em minutos), baseados no feedback da Nathor.\n",
    "SETUP_TIME_COLOR = 15\n",
    "SETUP_TIME_PIECE = 3\n",
    "\n",
    "# Capacidade produtiva diária total (em minutos).\n",
    "# Calculado com base no horário de funcionamento: 05h10 às 23h45.\n",
    "DAILY_CAPACITY_MINUTES = 18 * 60 + 35\n",
    "\n",
    "# Parâmetros físicos da linha de produção.\n",
    "# Origem: Reunião de alinhamento 2 (Sprint 4).\n",
    "MONOVIA_LENGHT_METERS = 168\n",
    "\n",
    "# --- 3. DADOS DE ENTRADA (Simulados a partir do PDF \"Dados Essenciais\") ---\n",
    "# Em um ambiente de produção real, estes dados seriam carregados de fontes\n",
    "# externas (planilhas, APIs, banco de dados). Para esta simulação, eles\n",
    "# são definidos estaticamente aqui, representando a \"fotografia\" do estado\n",
    "# da produção em um determinado momento.\n",
    "\n",
    "# REGRA DE NEGÓCIO: Lista de peças que necessitam de antecipação.\n",
    "# Componentes que passam por processos adicionais (ex: injeção) após a pintura.\n",
    "PECAS_COM_PROCESSO_ADICIONAL = ['206010040003.0'] # Exemplo: PÉ DE VELA\n",
    "DIAS_ANTECIPACAO = 2\n",
    "\n",
    "# RESTRIÇÃO DE CAPACIDADE: Fornecimento diário da Metalurgia.\n",
    "# Representa a quantidade máxima de cada peça que a metalurgia consegue fornecer\n",
    "# por dia para a pintura.\n",
    "FORNECIMENTO_METALURGIA = {\n",
    "    '206010180003.0': 1500, '208010180003.0': 1200,\n",
    "    '204010180002.0': 1000, '206010040003.0': 800\n",
    "}\n",
    "\n",
    "# RESTRIÇÃO DE CAPACIDADE: Armazenamento Pós-Pintura (Gaiolas).\n",
    "# Representa a quantidade máxima de cada peça pintada que pode ser armazenada\n",
    "# simultaneamente.\n",
    "CAPACIDADE_GAIOLAS = {\n",
    "    '206010180003.0': 2000, '208010180003.0': 1000,\n",
    "    '204010180002.0': 3000, '206010040003.0': 1500\n",
    "}\n",
    "\n",
    "# DADOS PRINCIPAIS: Ordens de Produção a serem planejadas.\n",
    "# Geração de uma massa de dados para simular um cenário com múltiplas ordens.\n",
    "print(\"Gerando massa de dados de produção para simulação...\")\n",
    "ordens_de_producao = []\n",
    "product_codes = list(FORNECIMENTO_METALURGIA.keys())\n",
    "cores = ['PRETO', 'AZUL', 'BRANCO', 'VERMELHO', 'AMARELO', 'VERDE']\n",
    "start_date = datetime.now()\n",
    "\n",
    "for _ in range(50):\n",
    "    code = random.choice(product_codes)\n",
    "    ordem = {\n",
    "        'CODIGO_PRODUTO': code, 'Tinta': random.choice(cores),\n",
    "        'Quantidade_Planejada': random.randint(300, 1600),\n",
    "        'Estoque': random.randint(0, 500),\n",
    "        'Data_de_Entrega': start_date + timedelta(days=random.randint(2, 10)),\n",
    "        'Tempo_Calculado': round(random.uniform(0.5, 1.5), 4),\n",
    "        'Pecas_por_Gancheira': random.randint(2, 8),\n",
    "        'ESTOQUE_GANCHEIRA': random.randint(100, 250),\n",
    "        'DISTANCIA_M': round(random.uniform(0.5, 1.2), 2)\n",
    "    }\n",
    "    ordem['Pedidos'] = ordem['Quantidade_Planejada']\n",
    "    ordens_de_producao.append(ordem)\n",
    "print(f\"{len(ordens_de_producao)} ordens de produção simuladas foram geradas.\")\n",
    "\n",
    "# --- 4. FUNÇÕES DE LÓGICA E OTIMIZAÇÃO ---\n",
    "\n",
    "def preprocessar_pedidos(lista_de_pedidos):\n",
    "    \"\"\"Aplica regras de negócio aos pedidos antes do início do planejamento.\n",
    "\n",
    "    Esta função atua como uma camada de pré-processamento, \"limpando\" e\n",
    "    ajustando os dados de entrada para que o algoritmo principal possa operar\n",
    "    de forma mais simples e direta. A principal regra implementada aqui é a\n",
    "    antecipação da data de entrega para peças que possuem processos\n",
    "    subsequentes à pintura, tornando-as artificialmente mais urgentes.\n",
    "\n",
    "    Args:\n",
    "        lista_de_pedidos (list): A lista original de dicionários, onde cada\n",
    "                                 dicionário representa uma ordem de produção.\n",
    "\n",
    "    Returns:\n",
    "        list: A lista de pedidos com as datas de entrega ajustadas conforme\n",
    "              as regras de negócio. Retorna uma cópia profunda para não\n",
    "              modificar a lista original.\n",
    "    \"\"\"\n",
    "    pedidos_ajustados = copy.deepcopy(lista_de_pedidos)\n",
    "    for item in pedidos_ajustados:\n",
    "        if item['CODIGO_PRODUTO'] in PECAS_COM_PROCESSO_ADICIONAL:\n",
    "            item['Data_de_Entrega'] -= timedelta(days=DIAS_ANTECIPACAO)\n",
    "            item['Observacao'] = f'Entrega antecipada em {DIAS_ANTECIPACAO} dias (proc. adicional)'\n",
    "    return pedidos_ajustados\n",
    "\n",
    "def calculate_cost(sequence):\n",
    "    \"\"\"Calcula o custo TOTAL de setup de uma determinada sequência de produção.\n",
    "\n",
    "    Esta é a função objetivo para o otimizador do Estágio 2. Ela quantifica\n",
    "    a eficiência de uma sequência medindo o tempo total perdido em setups.\n",
    "    A meta do otimizador é encontrar a permutação da sequência que minimiza o\n",
    "    valor retornado por esta função.\n",
    "\n",
    "    Args:\n",
    "        sequence (list): Uma lista de dicionários, representando uma sequência\n",
    "                         de produção para um dia.\n",
    "\n",
    "    Returns:\n",
    "        int: O custo total de setup em minutos, que é a soma de todas as\n",
    "             trocas de cor (15 min) e de peça (3 min) na sequência.\n",
    "    \"\"\"\n",
    "    total_setup_cost = 0\n",
    "    if len(sequence) < 2:\n",
    "        return 0\n",
    "    # Itera por todos os pares adjacentes de itens na sequência\n",
    "    for i in range(len(sequence) - 1):\n",
    "        # Adiciona custo se a cor mudar\n",
    "        if sequence[i]['Tinta'] != sequence[i+1]['Tinta']:\n",
    "            total_setup_cost += SETUP_TIME_COLOR\n",
    "        # Adiciona custo se o tipo de peça mudar\n",
    "        if sequence[i]['CODIGO_PRODUTO'] != sequence[i+1]['CODIGO_PRODUTO']:\n",
    "            total_setup_cost += SETUP_TIME_PIECE\n",
    "    return total_setup_cost\n",
    "\n",
    "def calculate_prioritization_score(item):\n",
    "    \"\"\"Calcula uma pontuação de prioridade para um item de produção.\n",
    "\n",
    "    Esta função é o núcleo da lógica de negócio do Estágio 1. Ela converte\n",
    "    as regras de prioridade da Nathor em uma pontuação matemática que permite\n",
    "    uma ordenação inequívoca de todas as ordens de produção. A ordenação é\n",
    "    feita em três níveis hierárquicos para desempate.\n",
    "\n",
    "    Args:\n",
    "        item (dict): O dicionário representando um item de produção.\n",
    "\n",
    "    Returns:\n",
    "        tuple: Uma tupla de 3 elementos. O Python ordena tuplas elemento por\n",
    "               elemento, o que implementa a hierarquia de prioridades:\n",
    "               1. Urgência (0 para urgente, 1 para normal)\n",
    "               2. Data de Entrega (em formato timestamp, menor é mais prioritário)\n",
    "               3. Tamanho do Déficit (negativo para priorizar o maior déficit)\n",
    "    \"\"\"\n",
    "    saldo = item['Estoque'] - item['Pedidos']\n",
    "    due_date = item['Data_de_Entrega']\n",
    "    \n",
    "    # Nível 1: Urgência. Itens com saldo negativo são sempre prioritários.\n",
    "    prioridade_urgente = 1 if saldo >= 0 else 0\n",
    "    # Nível 2: Data de Entrega. Desempata pela data mais próxima.\n",
    "    prioridade_data = due_date.timestamp()\n",
    "    # Nível 3: Tamanho do Déficit. Desempata pela maior necessidade de produção.\n",
    "    prioridade_saldo = abs(saldo) if saldo < 0 else 0\n",
    "    \n",
    "    return (prioridade_urgente, prioridade_data, -prioridade_saldo)\n",
    "\n",
    "def daily_sequencing_and_scheduling(unscheduled_items):\n",
    "    \"\"\"ESTÁGIO 1: Planeja a produção diária, aplicando todas as restrições.\n",
    "\n",
    "    Esta é a função mais complexa do sistema, responsável por simular o processo\n",
    "    de planejamento diário. Ela decide QUAIS itens serão produzidos em CADA dia.\n",
    "\n",
    "    O processo interno pode ser resumido em:\n",
    "    1. Filtra ordens que são permanentemente inviáveis (ex: falta de gancheiras).\n",
    "    2. Entra em um loop que simula cada dia de produção.\n",
    "    3. A cada dia, reseta os recursos diários (metalurgia, gaiolas).\n",
    "    4. Ordena todos os itens restantes pela prioridade de negócio.\n",
    "    5. Itera pela lista ordenada e tenta \"encaixar\" cada item no dia corrente,\n",
    "       passando por uma série de filtros de capacidade (metalurgia, gaiolas, tempo).\n",
    "    6. Itens que passam em todos os filtros são agendados para o dia. Os que\n",
    "       falham são guardados para serem avaliados no dia seguinte.\n",
    "    7. O loop termina quando não há mais itens para planejar ou quando os\n",
    "       itens restantes não cabem em nenhum dia.\n",
    "\n",
    "    Args:\n",
    "        unscheduled_items (list): A lista completa de itens a serem planejados,\n",
    "                                  já pré-processada.\n",
    "\n",
    "    Returns:\n",
    "        tuple: Uma tupla contendo duas listas:\n",
    "               - schedule (list): O cronograma de produção, onde cada elemento\n",
    "                 é um dicionário representando um dia.\n",
    "               - permanently_rejected (list): Lista de itens que não puderam\n",
    "                 ser planejados, com o motivo da rejeição.\n",
    "    \"\"\"\n",
    "    schedule = []\n",
    "    permanently_rejected = []\n",
    "    plannable_items = []\n",
    "\n",
    "    # Passo 1: Filtro de inviabilidade permanente. Remove itens que NUNCA\n",
    "    # poderiam ser planejados.\n",
    "    for item in unscheduled_items:\n",
    "        gancheiras_necessarias = math.ceil(item['Quantidade_Planejada'] / item['Pecas_por_Gancheira'])\n",
    "        comprimento_na_monovia = gancheiras_necessarias * item['DISTANCIA_M']\n",
    "        \n",
    "        # Lógica condicional de gancheiras, conforme reunião.\n",
    "        if comprimento_na_monovia < MONOVIA_LENGHT_METERS and gancheiras_necessarias > item['ESTOQUE_GANCHEIRA']:\n",
    "            item['Motivo_Rejeicao'] = f\"Gancheiras insuficientes ({gancheiras_necessarias} > {item['ESTOQUE_GANCHEIRA']})\"\n",
    "            permanently_rejected.append(item)\n",
    "        else:\n",
    "            plannable_items.append(item) # Item é considerado planejável\n",
    "\n",
    "    # Passo 2: Loop de planejamento diário sobre os itens planejáveis.\n",
    "    day_number = 1\n",
    "    while plannable_items:\n",
    "        # A cada novo dia, reseta os contadores de consumo de recursos diários.\n",
    "        consumo_metalurgia_diario = {code: 0 for code in FORNECIMENTO_METALURGIA}\n",
    "        consumo_gaiolas_diario = {code: 0 for code in CAPACIDADE_GAIOLAS}\n",
    "        \n",
    "        plannable_items.sort(key=calculate_prioritization_score)\n",
    "        \n",
    "        items_for_today = []\n",
    "        items_for_next_days = [] # Itens que não cabem hoje, mas podem caber amanhã.\n",
    "        time_used_today, last_color, last_piece_code = 0, None, None\n",
    "\n",
    "        # Itera sobre a lista de itens prioritários para tentar preencher o dia.\n",
    "        for item in plannable_items:\n",
    "            cod_produto = item['CODIGO_PRODUTO']\n",
    "            qtd_planejada = item['Quantidade_Planejada']\n",
    "            passes_daily_checks = True\n",
    "\n",
    "            # Filtro 2: Fornecimento da Metalurgia\n",
    "            if consumo_metalurgia_diario.get(cod_produto, 0) + qtd_planejada > FORNECIMENTO_METALURGIA.get(cod_produto, float('inf')):\n",
    "                passes_daily_checks = False\n",
    "            \n",
    "            # Filtro 3: Capacidade de Armazenamento (Gaiolas)\n",
    "            if consumo_gaiolas_diario.get(cod_produto, 0) + qtd_planejada > CAPACIDADE_GAIOLAS.get(cod_produto, float('inf')):\n",
    "                passes_daily_checks = False\n",
    "            \n",
    "            # Filtro 4: Capacidade de Tempo\n",
    "            item_time = item['Tempo_Calculado'] * 60\n",
    "            setup_cost = 0\n",
    "            if items_for_today:\n",
    "                if last_color != item['Tinta']: setup_cost += SETUP_TIME_COLOR\n",
    "                if last_piece_code != cod_produto: setup_cost += SETUP_TIME_PIECE\n",
    "            \n",
    "            if time_used_today + item_time + setup_cost > DAILY_CAPACITY_MINUTES:\n",
    "                passes_daily_checks = False\n",
    "            \n",
    "            if passes_daily_checks:\n",
    "                # Se o item passou em todos os filtros, é agendado para o dia.\n",
    "                items_for_today.append(item)\n",
    "                # E os recursos do dia são consumidos.\n",
    "                time_used_today += item_time + setup_cost\n",
    "                consumo_metalurgia_diario[cod_produto] += qtd_planejada\n",
    "                consumo_gaiolas_diario[cod_produto] += qtd_planejada\n",
    "                last_color, last_piece_code = item['Tinta'], cod_produto\n",
    "            else:\n",
    "                # Se falhou em algum filtro diário, vai para a lista do dia seguinte.\n",
    "                items_for_next_days.append(item)\n",
    "        \n",
    "        if not items_for_today:\n",
    "            # Se não foi possível agendar NENHUM item hoje, os restantes são\n",
    "            # considerados não planejáveis por falta de capacidade.\n",
    "            for item in items_for_next_days:\n",
    "                item['Motivo_Rejeicao'] = \"Não coube no cronograma (falta de capacidade diária)\"\n",
    "                permanently_rejected.append(item)\n",
    "            break # Encerra o planejamento.\n",
    "            \n",
    "        # Adiciona o dia preenchido ao cronograma final.\n",
    "        schedule.append({'day': day_number, 'items': items_for_today})\n",
    "        # A lista para o próximo dia de planejamento é a lista de itens que não couberam hoje.\n",
    "        plannable_items = items_for_next_days\n",
    "        day_number += 1\n",
    "        \n",
    "    return schedule, permanently_rejected\n",
    "\n",
    "def tabu_search_optimizer(daily_sequence, tabu_tenure=7, max_iterations=100):\n",
    "    \"\"\"ESTÁGIO 2: Otimiza a sequência de um único dia usando Busca Tabu.\"\"\"\n",
    "    # ... (A documentação desta função da resposta anterior já está completa)\n",
    "    current_solution = list(daily_sequence)\n",
    "    best_solution = list(daily_sequence)\n",
    "    best_cost = calculate_cost(best_solution)\n",
    "    tabu_list = []\n",
    "\n",
    "    for _ in range(max_iterations):\n",
    "        best_neighbor, best_neighbor_cost, best_move = None, float('inf'), None\n",
    "        for i in range(len(current_solution)):\n",
    "            for j in range(i + 1, len(current_solution)):\n",
    "                neighbor = list(current_solution)\n",
    "                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]\n",
    "                move = tuple(sorted((i, j)))\n",
    "                if move in tabu_list: continue\n",
    "                \n",
    "                neighbor_cost = calculate_cost(neighbor)\n",
    "                if neighbor_cost < best_neighbor_cost:\n",
    "                    best_neighbor, best_neighbor_cost, best_move = neighbor, neighbor_cost, move\n",
    "\n",
    "        if best_neighbor:\n",
    "            current_solution = best_neighbor\n",
    "            tabu_list.append(best_move)\n",
    "            if len(tabu_list) > tabu_tenure: tabu_list.pop(0)\n",
    "            if best_neighbor_cost < best_cost: best_solution, best_cost = best_neighbor, best_neighbor_cost\n",
    "    \n",
    "    return best_solution\n",
    "\n",
    "def export_to_excel(schedule, rejected_items, filename=\"cronograma_otimizado_sprint4_documentado.xlsx\"):\n",
    "    \"\"\"Exporta o resultado final para um arquivo Excel com duas abas.\"\"\"\n",
    "    # ... (A documentação desta função da resposta anterior já está completa)\n",
    "    with pd.ExcelWriter(filename, engine='openpyxl') as writer:\n",
    "        # Aba 1: Cronograma Otimizado\n",
    "        processed_data = []\n",
    "        for day_data in schedule:\n",
    "            for order, item in enumerate(day_data['items'], 1):\n",
    "                saldo = item['Estoque'] - item['Pedidos']\n",
    "                row = {'Dia': day_data['day'], 'Ordem_no_Dia': order, 'Saldo': saldo, **item}\n",
    "                processed_data.append(row)\n",
    "        \n",
    "        if processed_data:\n",
    "            df_schedule = pd.DataFrame(processed_data)\n",
    "            df_schedule['Data_de_Entrega'] = df_schedule['Data_de_Entrega'].dt.strftime('%Y-%m-%d')\n",
    "            column_order = [\n",
    "                'Dia', 'Ordem_no_Dia', 'CODIGO_PRODUTO', 'Tinta', 'Quantidade_Planejada',\n",
    "                'Estoque', 'Pedidos', 'Saldo', 'Data_de_Entrega', 'Tempo_Calculado',\n",
    "                'Pecas_por_Gancheira', 'ESTOQUE_GANCHEIRA', 'DISTANCIA_M', 'Observacao'\n",
    "            ]\n",
    "            df_schedule = df_schedule.reindex(columns=column_order)\n",
    "            df_schedule.to_excel(writer, sheet_name='Cronograma_Otimizado', index=False)\n",
    "            print(f\"\\nAba 'Cronograma_Otimizado' gerada com sucesso.\")\n",
    "\n",
    "        # Aba 2: Itens Não Planejados\n",
    "        if rejected_items:\n",
    "            df_rejected = pd.DataFrame(rejected_items)\n",
    "            if 'Data_de_Entrega' in df_rejected.columns:\n",
    "                df_rejected['Data_de_Entrega'] = df_rejected['Data_de_Entrega'].dt.strftime('%Y-%m-%d')\n",
    "            \n",
    "            rejected_cols = [\n",
    "                'CODIGO_PRODUTO', 'Tinta', 'Quantidade_Planejada',\n",
    "                'Pedidos', 'Data_de_Entrega', 'Motivo_Rejeicao'\n",
    "            ]\n",
    "            df_rejected = df_rejected.reindex(columns=rejected_cols)\n",
    "            df_rejected.to_excel(writer, sheet_name='Itens_Nao_Planejados', index=False)\n",
    "            print(f\"Aba 'Itens_Nao_Planejados' gerada com sucesso.\")\n",
    "\n",
    "    print(f\"\\nArquivo '{filename}' salvo com sucesso.\")\n",
    "\n",
    "\n",
    "# --- 5. EXECUÇÃO PRINCIPAL DO SCRIPT ---\n",
    "if __name__ == \"__main__\":\n",
    "    # O bloco __main__ é o ponto de entrada do script. Ele orquestra a chamada\n",
    "    # das funções na ordem correta para gerar o resultado final.\n",
    "    start_time = time.time()\n",
    "    \n",
    "    print(\"\\nPASSO 1: Aplicando regras de negócio (pré-processamento)...\")\n",
    "    pedidos_prontos = preprocessar_pedidos(ordens_de_producao)\n",
    "    \n",
    "    print(\"PASSO 2: Executando Estágio 1 (Planejamento e alocação diária)...\")\n",
    "    initial_schedule, rejected_items = daily_sequencing_and_scheduling(pedidos_prontos)\n",
    "    \n",
    "    print(\"PASSO 3: Executando Estágio 2 (Otimização da sequência de cada dia)...\")\n",
    "    optimized_schedule = []\n",
    "    for day_data in initial_schedule:\n",
    "        refined_sequence = tabu_search_optimizer(day_data['items'])\n",
    "        optimized_schedule.append({'day': day_data['day'], 'items': refined_sequence})\n",
    "    print(\"Otimização concluída.\")\n",
    "    \n",
    "    print(\"PASSO 4: Exportando resultados para o arquivo Excel...\")\n",
    "    export_to_excel(optimized_schedule, rejected_items)\n",
    "\n",
    "    end_time = time.time()\n",
    "    print(f\"\\nProcesso finalizado com sucesso em {end_time - start_time:.2f} segundos.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "37244f77",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "venv",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

NameError: name 'null' is not defined